# AutoGluon Classification Pipeline

* Author by Sawit Koseeyaumporn (Folk) - EXP 500042

* After install the autogluon make sure you restart the session before running the code

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/iot-network-attack/Sample_submission.csv
/kaggle/input/iot-network-attack/train.csv
/kaggle/input/iot-network-attack/test.csv
/kaggle/input/eeg-classification-supa/Sample_submission.csv
/kaggle/input/eeg-classification-supa/train.csv
/kaggle/input/eeg-classification-supa/test.csv


## 1. EDA (Exploratory Data Analysis)

In [ ]:
df = pd.read_csv("/kaggle/input/eeg-classification-supa/train.csv")
df.head()

* Check nulls

In [5]:
df.isnull().sum()

id          0
V0          0
V1          0
V2        514
V3       1264
V4          0
V5       1501
V6          0
V7          0
V8          0
V9        256
V10         0
V11         0
V12         0
V13         0
V14      2035
V15         0
Class       0
dtype: int64

* Deal with the null/NaN values with the KNNimputer

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)

columns_to_impute = ["V2", "V3", "V5", "V9", "V14"]

# Impute mean values
for col in columns_to_impute:
    df[col] = df[col].fillna(df[col].mean())


df

        id        V0        V1        V2        V3        V4        V5  \
0        0  0.292428  0.293124  0.314311  0.358699  0.331543  0.225766   
1        1  0.227376  0.371844  0.281638  0.296169  0.165798  0.183871   
2        2  0.630380  0.355120  0.373488  0.266282  0.339572  0.080644   
3        3  0.892770  0.385269  0.414566  0.353979  0.339572  0.338680   
4        4  0.430202  0.107202  0.463268  0.114090  0.235297  0.167705   
...    ...       ...       ...       ...       ...       ...       ...   
7483  7483  0.093832  0.309881  0.342845  0.339675  0.264703  0.199975   
7484  7484  0.986165  0.360150  0.369389  0.288008  0.334202  0.212870   
7485  7485  0.633770  0.345060  0.406121  0.345080  0.368978  0.264515   
7486  7486  0.827227  0.653275  0.414566  0.353979  0.585558  0.293515   
7487  7487  0.061330  0.164168  0.238777  0.290764  0.229927  0.233684   

            V6        V7        V8        V9       V10       V11       V12  \
0     0.239241  0.272733  0.28661

## Model training

In [7]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# Specify the column to predict (target variable)
label_column = "Class"

# Initialize the AutoGluon TabularPredictor
predictor = TabularPredictor(label=label_column, eval_metric="accuracy").fit(df, time_limit=300)

No path specified. Models will be saved in: "AutogluonModels/ag-20250423_014632"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.18 GB / 31.35 GB (96.3%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions a

[1000]	valid_set's binary_error: 0.136182


	0.8638	 = Validation score   (accuracy)
	6.2s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 292.82s of the 292.82s of remaining time.


[1000]	valid_set's binary_error: 0.0881175


	0.9146	 = Validation score   (accuracy)
	2.23s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ... Training model for up to 290.50s of the 290.50s of remaining time.
	0.8985	 = Validation score   (accuracy)
	2.34s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestEntr ... Training model for up to 288.00s of the 288.00s of remaining time.
	0.9025	 = Validation score   (accuracy)
	2.83s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 285.03s of the 285.03s of remaining time.
	0.9306	 = Validation score   (accuracy)
	13.43s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ... Training model for up to 271.59s of the 271.59s of remaining time.
	0.8812	 = Validation score   (accuracy)
	1.0s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: ExtraTreesEntr ... Training model for up to 270.35s of the 270.35s of remaining time.
	0.

[1000]	valid_set's binary_error: 0.0867824


	0.9159	 = Validation score   (accuracy)
	5.31s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 299.92s of the 208.31s of remaining time.
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.9346	 = Validation score   (accuracy)
	0.1s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 91.83s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 27479.0 rows/s (749 batch size)
Disabling decision threshold calibration for metric `accuracy` due to having fewer than 10000 rows of validation data for calibration, to avoid overfitting (749 rows).
	`accuracy` is generally not improved through threshold calibration. Force calibration via specifying `calibrate_decision_threshold=True`.
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/kaggle/working/AutogluonModels/ag-20250423_014632")


In [8]:
test_df = pd.read_csv("/kaggle/input/eeg-classification-supa/test.csv")

test_df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,Class
0,0,0.817868,0.319941,0.377547,0.372264,0.360949,0.212870,0.276878,0.319874,0.423668,0.339272,0.083986,0.347192,0.443717,0.460898,NaN,0.418423,NaN
1,1,0.130386,0.350090,0.540831,0.364104,0.425130,NaN,0.104838,0.393933,0.146407,0.217283,0.450890,0.354534,0.304636,0.437487,NaN,0.400283,NaN
2,2,0.679447,0.269672,0.348973,0.353240,0.270073,0.161288,0.284950,0.195325,0.289715,0.264871,0.404879,0.281158,0.386294,0.390615,0.409448,0.386331,NaN
3,3,0.034554,0.366846,0.504099,0.353240,0.296767,0.283827,0.325313,0.612829,0.323978,0.357147,0.293370,0.378993,0.529809,0.424487,0.451442,0.428187,NaN
4,4,0.934683,0.262976,0.328598,NaN,0.248644,0.219350,0.279604,0.289607,0.199380,0.244037,0.093934,0.273863,0.269295,0.315103,0.345139,0.347276,NaN


In [9]:
test_df.isnull().sum()

id          0
V0          0
V1          0
V2        487
V3       1237
V4          0
V5       1500
V6          0
V7          0
V8          0
V9        245
V10         0
V11         0
V12         0
V13         0
V14      1966
V15         0
Class    7488
dtype: int64

In [10]:
# List of columns to impute
columns_to_impute = ["V2", "V3", "V5", "V9", "V14"]

# Impute mean values
for col in columns_to_impute:
    test_df[col] = test_df[col].fillna(test_df[col].mean())

print(test_df)

        id        V0        V1        V2        V3        V4        V5  \
0        0  0.817868  0.319941  0.377547  0.372264  0.360949  0.212870   
1        1  0.130386  0.350090  0.540831  0.364104  0.425130  0.229927   
2        2  0.679447  0.269672  0.348973  0.353240  0.270073  0.161288   
3        3  0.034554  0.366846  0.504099  0.353240  0.296767  0.283827   
4        4  0.934683  0.262976  0.328598  0.352162  0.248644  0.219350   
...    ...       ...       ...       ...       ...       ...       ...   
7483  7483  0.209261  0.273036  0.418378  0.252716  0.318196  0.209662   
7484  7484  0.661979  0.355120  0.320400  0.366859  0.318196  0.267723   
7485  7485  0.688826  0.376874  0.434694  0.266282  0.347602  0.212870   
7486  7486  0.169658  0.313245  0.395933  0.410312  0.358290  0.229927   
7487  7487  0.784474  0.606369  0.665314  0.695671  0.668457  0.609675   

            V6        V7        V8        V9       V10       V11       V12  \
0     0.276878  0.319874  0.42366

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [11]:
test_df.drop('Class', inplace=True, axis=1)

In [12]:
test_df.head()

,id,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,0,0.817868,0.319941,0.377547,0.372264,0.360949,0.212870,0.276878,0.319874,0.423668,0.339272,0.083986,0.347192,0.443717,0.460898,0.416573,0.418423
1,1,0.130386,0.350090,0.540831,0.364104,0.425130,0.229927,0.104838,0.393933,0.146407,0.217283,0.450890,0.354534,0.304636,0.437487,0.416573,0.400283
2,2,0.679447,0.269672,0.348973,0.353240,0.270073,0.161288,0.284950,0.195325,0.289715,0.264871,0.404879,0.281158,0.386294,0.390615,0.409448,0.386331
3,3,0.034554,0.366846,0.504099,0.353240,0.296767,0.283827,0.325313,0.612829,0.323978,0.357147,0.293370,0.378993,0.529809,0.424487,0.451442,0.428187
4,4,0.934683,0.262976,0.328598,0.352162,0.248644,0.219350,0.279604,0.289607,0.199380,0.244037,0.093934,0.273863,0.269295,0.315103,0.345139,0.347276


In [13]:
# Make predictions on the test data
predictions = predictor.predict(test_df)

# Save the predictions to a CSV file
output_path = "predictions.csv"
predictions.to_csv(output_path)
print(f"Predictions saved to {output_path}")

Predictions saved to predictions.csv


In [14]:
predictions.value_counts()

Class
0    4150
1    3338
Name: count, dtype: int64

---

## IoT Network Attack Detection

In [15]:
import pandas as pd

df = pd.read_csv("/kaggle/input/iot-network-attack/train.csv")
df.head()

,id,id.orig_p,id.resp_p,proto,service,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
0,0,43296,53,udp,dns,0.020938,2,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Thing_Speak
1,1,33859,21,tcp,NaN,0.000004,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
2,2,64606,21,tcp,NaN,0.000004,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
3,3,54357,137,udp,dns,0.000000,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,ARP_poisioning
4,4,51036,1100,tcp,NaN,0.000006,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,1024,NMAP_OS_DETECTION


In [16]:
df.drop('service', inplace=True, axis=1)

In [17]:
df.describe()

,id,id.orig_p,id.resp_p,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,...,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size
count,61558.000000,61558.000000,61558.000000,61558.000000,61558.000000,61558.000000,61558.000000,61558.000000,6.155800e+04,6.155800e+04,...,6.155800e+04,6.155800e+04,6.155800e+04,6.155800e+04,6.155800e+04,6.155800e+04,6.155800e+04,61558.000000,61558.000000,61558.000000
mean,30778.500000,34698.806930,1028.740131,3.449925,2.166136,1.821973,1.396894,0.727542,3.524373e+05,3.524028e+05,...,1.497782e+05,2.047288e+04,1.620236e+06,1.707280e+06,3.194090e+06,1.669391e+06,4.556553e+04,6112.347705,2741.925550,743.779736
std,17770.408272,19043.235091,5284.921627,114.932118,13.461114,13.629044,10.894490,10.865317,3.713823e+05,3.714140e+05,...,1.853807e+06,7.543505e+05,8.857208e+06,9.279385e+06,1.106498e+08,9.046910e+06,1.041597e+06,18706.764815,10020.171671,6271.073732
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,15389.250000,17777.500000,21.000000,0.000001,1.000000,1.000000,1.000000,0.000000,7.311433e+01,7.121755e+01,...,9.536740e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
50%,30778.500000,37353.000000,21.000000,0.000004,1.000000,1.000000,1.000000,0.000000,2.467238e+05,2.467238e+05,...,4.053116e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
75%,46167.750000,50936.000000,21.000000,0.000005,1.000000,1.000000,1.000000,0.000000,5.242880e+05,5.242880e+05,...,5.006790e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,64.000000,0.000000,64.000000
max,61557.000000,65535.000000,65389.000000,18761.401290,1887.000000,1986.000000,1887.000000,1284.000000,1.048576e+06,1.048576e+06,...,4.374931e+08,1.767556e+08,3.000000e+08,3.000000e+08,1.864431e+10,3.000000e+08,1.192656e+08,65535.000000,65535.000000,65535.000000


In [18]:
df.columns

Index(['id', 'id.orig_p', 'id.resp_p', 'proto', 'flow_duration',
       'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
       'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
       'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot',
       'fwd_header_size_min', 'fwd_header_size_max', 'bwd_header_size_tot',
       'bwd_header_size_min', 'bwd_header_size_max', 'flow_FIN_flag_count',
       'flow_SYN_flag_count', 'flow_RST_flag_count', 'fwd_PSH_flag_count',
       'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_URG_flag_count',
       'bwd_URG_flag_count', 'flow_CWR_flag_count', 'flow_ECE_flag_count',
       'fwd_pkts_payload.min', 'fwd_pkts_payload.max', 'fwd_pkts_payload.tot',
       'fwd_pkts_payload.avg', 'fwd_pkts_payload.std', 'bwd_pkts_payload.min',
       'bwd_pkts_payload.max', 'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg',
       'bwd_pkts_payload.std', 'flow_pkts_payload.min',
       'flow_pkts_payload.max', 'flow_pkts_payload.tot',
       'flow_

In [19]:
mapping_dict = {'tcp': 0, 'udp': 1, 'icmp': 2}

df["proto"] = df["proto"].map(mapping_dict)

In [20]:
!pip install featurewiz

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 8.6 MB/s eta 0:00:00
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2435 sha256=ed3cfd54db2e54d9b87510aa01f9e902fc4347f9a608c4231954aaf747b6deb3
  Stored in directory: /root/.cache/pip/wheels/a6/ff/5d/4f10de26fe5ddef243c97f13c6cf579d7353d659e41a05c3a6
Successfully built feather-format
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
  Attempting uninstall: category_encoders
    Found existing installation: category_encoders 2.7.0
    Uninstalling category_encoders-2.7.0:
      Successfully uninstalled category_

## Featurewiz

In [21]:
from featurewiz import featurewiz

target = 'Attack_type'
feats = featurewiz(df, target, corr_limit=0.7, verbose=0)
len(feats)

Imported lazytransform v1.18. 

Imported featurewiz 0.6.1. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    
featurewiz has selected 0.7 as the correlation limit. Change this limit to fit your needs...
    Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
    Single_Label Multi_Classification problem 
    Loaded train data. Shape = (61558, 84)
    Some column names had special characters which were removed...
    Single_Label Multi_Classification problem 
No test data filename given...
Classifying features using a random sample of 10000 rows from dataset...

2

In [22]:
selected_features = ['id.resp_p', 'fwd_last_window_size', 'bwd_pkts_payload.avg', 'active.max', 'flow_iat.min', 'fwd_URG_flag_count', 'fwd_pkts_payload.avg', 'flow_pkts_payload.tot', 'fwd_header_size_tot', 'flow_FIN_flag_count', 'fwd_subflow_bytes', 'fwd_pkts_payload.tot', 'proto', 'fwd_pkts_per_sec', 'active.tot', 'flow_SYN_flag_count', 'bwd_iat.max', 'bwd_subflow_bytes', 'down_up_ratio', 'flow_RST_flag_count', 'flow_ECE_flag_count', 'fwd_bulk_packets']
final_feats = ['id.resp_p', 'fwd_last_window_size', 'bwd_pkts_payload.avg', 'active.max', 'flow_iat.min', 'fwd_URG_flag_count', 'fwd_pkts_payload.avg', 'flow_pkts_payload.tot', 'fwd_header_size_tot', 'flow_FIN_flag_count', 'fwd_subflow_bytes', 'fwd_pkts_payload.tot', 'proto', 'fwd_pkts_per_sec', 'active.tot', 'flow_SYN_flag_count', 'bwd_iat.max', 'bwd_subflow_bytes', 'down_up_ratio', 'flow_RST_flag_count', 'flow_ECE_flag_count', 'fwd_bulk_packets', 'Attack_type']

In [23]:
df

,id,id.orig_p,id.resp_p,proto,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
0,0,43296,53,1,0.020938,2,2,2,2,9.551939e+01,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Thing_Speak
1,1,33859,21,0,0.000004,1,1,1,0,2.467238e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
2,2,64606,21,0,0.000004,1,1,1,0,2.621440e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
3,3,54357,137,1,0.000000,1,0,1,0,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,ARP_poisioning
4,4,51036,1100,0,0.000006,1,1,0,0,1.677722e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,1024,NMAP_OS_DETECTION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61553,61553,36242,6502,0,0.000029,1,1,0,0,3.437954e+04,...,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,1024,NMAP_XMAS_TREE_SCAN
61554,61554,10427,21,0,0.000001,1,1,1,0,1.048576e+06,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
61555,61555,11395,21,0,0.000001,1,1,1,0,8.388608e+05,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,DOS_SYN_Hping
61556,61556,57712,443,0,1.901634,14,14,5,6,7.362090e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,64240,42600,0,ARP_poisioning


In [24]:
df['proto'].value_counts()

proto
0    55208
1     6319
2       31
Name: count, dtype: int64

In [25]:
df.columns

Index(['id', 'id.orig_p', 'id.resp_p', 'proto', 'flow_duration',
       'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
       'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
       'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot',
       'fwd_header_size_min', 'fwd_header_size_max', 'bwd_header_size_tot',
       'bwd_header_size_min', 'bwd_header_size_max', 'flow_FIN_flag_count',
       'flow_SYN_flag_count', 'flow_RST_flag_count', 'fwd_PSH_flag_count',
       'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_URG_flag_count',
       'bwd_URG_flag_count', 'flow_CWR_flag_count', 'flow_ECE_flag_count',
       'fwd_pkts_payload.min', 'fwd_pkts_payload.max', 'fwd_pkts_payload.tot',
       'fwd_pkts_payload.avg', 'fwd_pkts_payload.std', 'bwd_pkts_payload.min',
       'bwd_pkts_payload.max', 'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg',
       'bwd_pkts_payload.std', 'flow_pkts_payload.min',
       'flow_pkts_payload.max', 'flow_pkts_payload.tot',
       'flow_

In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Select only numerical columns
# numerical_cols = df.select_dtypes(include=['number']).columns

# Apply StandardScaler
scaler = StandardScaler()
df[selected_features] = scaler.fit_transform(df[selected_features])

In [27]:
df = df[final_feats]

In [28]:
df.head()

,id.resp_p,fwd_last_window_size,bwd_pkts_payload.avg,active.max,flow_iat.min,fwd_URG_flag_count,fwd_pkts_payload.avg,flow_pkts_payload.tot,fwd_header_size_tot,flow_FIN_flag_count,...,fwd_pkts_per_sec,active.tot,flow_SYN_flag_count,bwd_iat.max,bwd_subflow_bytes,down_up_ratio,flow_RST_flag_count,flow_ECE_flag_count,fwd_bulk_packets,Attack_type
0,-0.184629,-0.118606,0.400653,-0.064749,-0.016360,-0.128764,-1.419397,-0.032421,-0.121744,-0.243072,...,-0.948738,-0.018805,-2.004132,-0.090366,-0.02423,0.432142,-1.817370,-0.023813,-0.016157,Thing_Speak
1,-0.190684,-0.108400,-0.226539,-0.073447,-0.017693,-0.128764,0.429465,-0.036806,-0.108301,-0.243072,...,-0.284651,-0.020473,0.103430,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,DOS_SYN_Hping
2,-0.190684,-0.108400,-0.226539,-0.073447,-0.017694,-0.128764,0.429465,-0.036806,-0.108301,-0.243072,...,-0.243130,-0.020473,0.103430,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,DOS_SYN_Hping
3,-0.168734,-0.118606,-0.226539,-0.073449,-0.017714,-0.128764,-1.111254,-0.042286,-0.148630,-0.243072,...,-0.948996,-0.020474,-2.004132,-0.094939,-0.05237,-2.521718,-1.817370,-0.023813,-0.016157,ARP_poisioning
4,0.013484,0.044685,-0.226539,-0.073446,-0.017683,-0.128764,-2.211767,-0.046200,-0.108301,-0.243072,...,-0.497241,-0.020473,-2.004132,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,NMAP_OS_DETECTION


In [29]:
df['Attack_type'].value_counts()

Attack_type
DOS_SYN_Hping                 47329
Thing_Speak                    4054
ARP_poisioning                 3875
MQTT_Publish                   2073
NMAP_UDP_SCAN                  1295
NMAP_XMAS_TREE_SCAN            1005
NMAP_OS_DETECTION              1000
NMAP_TCP_scan                   501
DDOS_Slowloris                  267
Wipro_bulb                      126
Metasploit_Brute_Force_SSH       19
NMAP_FIN_SCAN                    14
Name: count, dtype: int64

In [30]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Attack_type'] = le.fit_transform(df['Attack_type'])

In [31]:
df.head()

,id.resp_p,fwd_last_window_size,bwd_pkts_payload.avg,active.max,flow_iat.min,fwd_URG_flag_count,fwd_pkts_payload.avg,flow_pkts_payload.tot,fwd_header_size_tot,flow_FIN_flag_count,...,fwd_pkts_per_sec,active.tot,flow_SYN_flag_count,bwd_iat.max,bwd_subflow_bytes,down_up_ratio,flow_RST_flag_count,flow_ECE_flag_count,fwd_bulk_packets,Attack_type
0,-0.184629,-0.118606,0.400653,-0.064749,-0.016360,-0.128764,-1.419397,-0.032421,-0.121744,-0.243072,...,-0.948738,-0.018805,-2.004132,-0.090366,-0.02423,0.432142,-1.817370,-0.023813,-0.016157,10
1,-0.190684,-0.108400,-0.226539,-0.073447,-0.017693,-0.128764,0.429465,-0.036806,-0.108301,-0.243072,...,-0.284651,-0.020473,0.103430,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,2
2,-0.190684,-0.108400,-0.226539,-0.073447,-0.017694,-0.128764,0.429465,-0.036806,-0.108301,-0.243072,...,-0.243130,-0.020473,0.103430,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,2
3,-0.168734,-0.118606,-0.226539,-0.073449,-0.017714,-0.128764,-1.111254,-0.042286,-0.148630,-0.243072,...,-0.948996,-0.020474,-2.004132,-0.094939,-0.05237,-2.521718,-1.817370,-0.023813,-0.016157,0
4,0.013484,0.044685,-0.226539,-0.073446,-0.017683,-0.128764,-2.211767,-0.046200,-0.108301,-0.243072,...,-0.497241,-0.020473,-2.004132,-0.094939,-0.05237,0.432142,0.465813,-0.023813,-0.016157,6


In [32]:
test_df = pd.read_csv('/kaggle/input/iot-network-attack/test.csv')

In [33]:
test_df.head()

,id,id.orig_p,id.resp_p,proto,service,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,Attack_type
0,0,41489,21,tcp,NaN,0.000005,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,NaN
1,1,49260,21,tcp,NaN,0.000001,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,NaN
2,2,42887,21,tcp,NaN,0.000004,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,NaN
3,3,36502,21,tcp,NaN,0.000005,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,64,0,64,NaN
4,4,36242,1309,tcp,NaN,0.000011,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1024,0,1024,NaN


In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

test_df.drop('service', inplace=True, axis=1)
test_df["proto"] = test_df["proto"].map(mapping_dict)

# Apply StandardScaler
scaler = StandardScaler()
test_df[selected_features] = scaler.fit_transform(test_df[selected_features])

In [35]:
test_df = test_df[selected_features]

In [36]:
test_df

,id.resp_p,fwd_last_window_size,bwd_pkts_payload.avg,active.max,flow_iat.min,fwd_URG_flag_count,fwd_pkts_payload.avg,flow_pkts_payload.tot,fwd_header_size_tot,flow_FIN_flag_count,...,proto,fwd_pkts_per_sec,active.tot,flow_SYN_flag_count,bwd_iat.max,bwd_subflow_bytes,down_up_ratio,flow_RST_flag_count,flow_ECE_flag_count,fwd_bulk_packets
0,-0.187249,-0.109546,-0.223568,-0.051335,-0.016869,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,-0.409155,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
1,-0.187249,-0.109546,-0.223568,-0.051336,-0.016882,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,1.317554,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
2,-0.187249,-0.109546,-0.223568,-0.051335,-0.016872,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,-0.282191,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
3,-0.187249,-0.109546,-0.223568,-0.051335,-0.016869,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,-0.409155,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
4,0.059134,0.041657,-0.223568,-0.051333,-0.016849,7.774068,-2.150897,-0.014795,-0.075746,1.861656,...,-0.338082,-0.702414,-0.020389,-2.005770,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61554,-0.175963,9.998435,-0.223568,-0.051337,-0.016886,-0.128633,-2.150897,-0.014795,-0.033144,-0.243677,...,-0.338082,-0.948752,-0.020390,0.103765,-0.095206,-0.023996,-2.540405,-1.827793,-0.020869,-0.016365
61555,0.168935,-0.040560,-0.152184,0.541865,-0.016634,-0.128633,-1.984061,-0.012876,0.580321,-0.243677,...,-0.338082,-0.948751,0.108792,2.213300,0.268520,-0.022804,-0.758579,0.465760,-0.020869,-0.016365
61556,-0.187249,-0.109546,-0.223568,-0.051335,-0.016872,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,-0.282191,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365
61557,-0.187249,-0.109546,-0.223568,-0.051336,-0.016879,-0.128633,0.415814,-0.012775,-0.075746,-0.243677,...,-0.338082,0.467689,-0.020390,0.103765,-0.095206,-0.023996,0.429305,0.465760,-0.020869,-0.016365


In [37]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# Specify the column to predict (target variable)
label_column = "Attack_type"

# Initialize the AutoGluon TabularPredictor
predictor = TabularPredictor(label=label_column, eval_metric="accuracy").fit(df, time_limit=300)

No path specified. Models will be saved in: "AutogluonModels/ag-20250423_014912"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.28 GB / 31.35 GB (93.4%)
Disk Space Avail:   19.27 GB / 19.52 GB (98.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions a

In [38]:
# Make predictions on the test data
predictions = predictor.predict(test_df)

# Save the predictions to a CSV file
output_path = "iot_predictions.csv"
predictions.to_csv(output_path)
print(f"Predictions saved to {output_path}")

Predictions saved to iot_predictions.csv


In [39]:
# Load the test predictions
predictions_df = pd.read_csv("iot_predictions.csv", index_col=0) # Assuming the first column is the index

predictions_df

,Attack_type
0,2
1,2
2,2
3,2
4,10
...,...
61554,10
61555,10
61556,2
61557,2


In [40]:
# Now, use the inverse_transform method to map the numerical predictions back to their original labels
# Assuming your predictions are in a column named 'predicted_attack_type' in predictions_df
predictions_df['Attack_type'] = le.inverse_transform(predictions_df['Attack_type'])

# Save the predictions with the original labels (optional)
output_path_with_original = "iot_predictions_with_original_labels.csv"
predictions_df.to_csv(output_path_with_original)
print(f"Predictions with original labels saved to {output_path_with_original}")

Predictions with original labels saved to iot_predictions_with_original_labels.csv


In [41]:
predictions = inverse_transform_labels(predictions, le, 'Attack_type')

NameError: name 'inverse_transform_labels' is not defined